<a href="https://colab.research.google.com/github/PlayfulDevBit/Stock-analysis/blob/main/Technical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install required libraries
!pip install yfinance transformers gspread pandas


In [3]:

from google.colab import auth
from google.auth import default
import gspread
import yfinance as yf
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from datetime import datetime, timedelta

# Authenticate user (prompts for Google account login and permission)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet (replace 'Your_Sheet_Name' with your sheet's name)
sheet = gc.open("Stock_values").worksheet("Sheet1")  # Replace with your sheet and tab name

# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

def calculate_sma(prices, window):
    """Calculate Simple Moving Average"""
    if len(prices) < window:
        return None
    return np.mean(prices[-window:])

def get_technical_decision(ticker):
    """Perform technical analysis for a 3-5 year investment window"""
    try:
        # Download 5 years of monthly stock data
        stock = yf.Ticker(ticker)
        data = stock.history(period="5y", interval="1mo")

        if data.empty:
            return "ERROR: No data"

        # Get closing prices
        prices = data["Close"].dropna().values

        if len(prices) < 50:
            return "ERROR: Insufficient data"

        # Calculate SMAs
        sma50 = calculate_sma(prices, 50)
        sma200 = calculate_sma(prices, 200) if len(prices) >= 200 else None
        current_price = prices[-1]

        # Calculate 3-year price trend
        three_year_idx = max(0, len(prices) - 36)
        three_year_price = prices[three_year_idx]

        # Technical analysis logic
        decision = "KEEP"
        if sma50 and sma200:
            if sma50 > sma200 and current_price > sma50:
                decision = "BUY"
            elif sma50 < sma200 and current_price < sma50:
                decision = "SELL"

        # Fallback: 3-year price trend
        if decision == "KEEP" and three_year_price:
            if current_price > three_year_price * 1.2:  # 20% growth
                decision = "BUY"
            elif current_price < three_year_price * 0.8:  # 20% decline
                decision = "SELL"

        return decision

    except Exception as e:
        return f"ERROR: {str(e)}"

def get_finbert_sentiment(text):
    """Analyze sentiment using FinBERT"""
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment_scores = probs.detach().numpy()[0]
        labels = ["positive", "negative", "neutral"]
        sentiment = labels[np.argmax(sentiment_scores)]
        return sentiment
    except Exception as e:
        return f"ERROR: {str(e)}"

def main():
    # Read stock quotes from column A (starting from A2)
    quotes = sheet.col_values(1)[1:]  # Skip header
    decisions = []

    for quote in quotes:
        if not quote:
            decisions.append("ERROR: Empty quote")
            continue

        # Perform technical analysis
        decision = get_technical_decision(quote)

        # Optional: Integrate FinBERT sentiment (placeholder for news)
        news_text = f"Sample news about {quote}"  # Replace with actual news text or API call
        sentiment = get_finbert_sentiment(news_text)

        # Combine technical and sentiment analysis
        if sentiment == "positive" and decision == "KEEP":
            decision = "BUY"
        elif sentiment == "negative" and decision == "KEEP":
            decision = "SELL"

        decisions.append(decision)

    # Write decisions to column R (R2 onward)
    for i, decision in enumerate(decisions, start=2):
        sheet.update_cell(i, 18, decision)  # Column R is 18
        time.sleep(1)  # Avoid Google Sheets API rate limits

if __name__ == "__main__":
    import time
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]